In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [8]:
# Load environment variables (ensure GROQ_API_KEY is set in your .env file)
load_dotenv()

# Initialize ChatGroq LLM
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="gemma2-9b-it")

In [9]:
# Text from Shahid Bhagat Singh (excerpt from "Why I Am An Atheist")
bhagat_singh_text = """
The more I think, the more I realize that there is no God who created this world or who directs it. History records many instances of people who have believed in God and done great deeds, but it also records many instances of people who have been atheists and done equally great deeds.

I refuse to believe in an all-powerful, all-knowing, all-just, and all-merciful God who allows the suffering and injustice in the world. If there is such a God, he must be either powerless or malicious. I cannot accept either of these conclusions.

My atheism is not an accidental happening. It is the result of prolonged study and thought. I have read the scriptures, pondered over the arguments for and against the existence of God, and come to the conclusion that there is no rational basis for belief in God.

I believe in humanity, in its potential for progress and liberation. I believe that it is through human effort, human struggle, and human sacrifice that a better world can be created. We do not need divine intervention to achieve our goals. We need courage, conviction, and a scientific approach.

Let us rise above sectarian quarrels and work for the common good. Let us strive to build a society where exploitation ceases to exist, where every individual has the opportunity to develop their full potential. This is the true religion, the true worship.
"""

In [10]:
bhagat_singh_text

'\nThe more I think, the more I realize that there is no God who created this world or who directs it. History records many instances of people who have believed in God and done great deeds, but it also records many instances of people who have been atheists and done equally great deeds.\n\nI refuse to believe in an all-powerful, all-knowing, all-just, and all-merciful God who allows the suffering and injustice in the world. If there is such a God, he must be either powerless or malicious. I cannot accept either of these conclusions.\n\nMy atheism is not an accidental happening. It is the result of prolonged study and thought. I have read the scriptures, pondered over the arguments for and against the existence of God, and come to the conclusion that there is no rational basis for belief in God.\n\nI believe in humanity, in its potential for progress and liberation. I believe that it is through human effort, human struggle, and human sacrifice that a better world can be created. We do 

In [11]:
# Prepare the chat messages
chat_message_bhagat_singh = [
    SystemMessage(content="You are an expert with expertise in summarizing speeches and philosophical texts."),
    HumanMessage(content=f"Please provide a short and concise summary of the following text:\n Text:{bhagat_singh_text}")
]


In [12]:
# Get the number of tokens in the Bhagat Singh text
tokens_bhagat_singh = llm.get_num_tokens(bhagat_singh_text)
print(f"Number of tokens in Bhagat Singh's text: {tokens_bhagat_singh}")

Number of tokens in Bhagat Singh's text: 286


In [13]:
# Get the summary using the direct LLM call
print("\n--- Summary of Bhagat Singh's Text ---")
response = llm(chat_message_bhagat_singh)
print(response.content)

print("\n--- Full LLM Response (AIMessage) ---")
print(response)


--- Summary of Bhagat Singh's Text ---
This text presents a passionate argument for atheism. The author rejects the existence of God based on the following points:

* **Inconsistency with observed suffering:**  The author finds it illogical to believe in a benevolent and omnipotent God who permits the world's injustice and pain. 
* **Lack of rational basis:** The author claims their atheism stems from careful study and reflection, finding no convincing evidence for God's existence.
* **Faith in humanity:** The author places their faith in human potential for progress and liberation, advocating for human effort and scientific inquiry to create a better world.

Ultimately, the text calls for a secular, humanistic approach to life, emphasizing social justice, individual development, and collective action over religious dogma. 





--- Full LLM Response (AIMessage) ---
content="This text presents a passionate argument for atheism. The author rejects the existence of God based on the foll

In [14]:
# Example of accessing output tokens from the response metadata
if hasattr(response, 'response_metadata') and 'token_usage' in response.response_metadata:
    input_tokens = response.response_metadata['token_usage'].get('prompt_tokens')
    output_tokens = response.response_metadata['token_usage'].get('completion_tokens')
    print(f"\nInput Tokens (from response metadata): {input_tokens}")
    print(f"Output Tokens (from response metadata): {output_tokens}")


Input Tokens (from response metadata): 315
Output Tokens (from response metadata): 153


# LLMChain with Prompttemplate

In [15]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

generictemplate="""
Write a summary of the following speech:
Speech:{speech}
Translate the precise summary to {language}
"""

prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generictemplate
)
prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template='\nWrite a summary of the following speech:\nSpeech:{speech}\nTranslate the precise summary to {language}\n')

In [19]:

complete_prompt=prompt.format(speech=bhagat_singh_text,language="French")
complete_prompt

'\nWrite a summary of the following speech:\nSpeech:\nThe more I think, the more I realize that there is no God who created this world or who directs it. History records many instances of people who have believed in God and done great deeds, but it also records many instances of people who have been atheists and done equally great deeds.\n\nI refuse to believe in an all-powerful, all-knowing, all-just, and all-merciful God who allows the suffering and injustice in the world. If there is such a God, he must be either powerless or malicious. I cannot accept either of these conclusions.\n\nMy atheism is not an accidental happening. It is the result of prolonged study and thought. I have read the scriptures, pondered over the arguments for and against the existence of God, and come to the conclusion that there is no rational basis for belief in God.\n\nI believe in humanity, in its potential for progress and liberation. I believe that it is through human effort, human struggle, and human s

In [20]:
llm.get_num_tokens(complete_prompt)

308

In [21]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run({'speech':bhagat_singh_text,'language':'hindi'})
summary

C:\Users\ajaym\AppData\Local\Temp\ipykernel_51740\1144084796.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(llm=llm,prompt=prompt)
C:\Users\ajaym\AppData\Local\Temp\ipykernel_51740\1144084796.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary=llm_chain.run({'speech':bhagat_singh_text,'language':'hindi'})


'## भाषण का सारांश \n\nइस भाषण में, वक्ता अपने नास्तिक दृष्टिकोण की व्याख्या करता है। वह मानता है कि प्रकृति का कोई निर्माता या निर्देशक नहीं है।  वह इतिहास के उदाहरणों का हवाला देते हुए कहते हैं कि धर्म से प्रेरित लोगों ने भी उतने ही महान कार्यों को किया है जितने नास्तिकों ने किया है।  \n\nवह एक सर्वशक्तिमान, सर्वज्ञ, सर्वन्यायपूर्ण और सर्वदयालु भगवान के अस्तित्व को अस्वीकार करता है जो दुनिया में दुख और अन्याय को बर्दाश्त करता है।  वह मानता है कि अगर ऐसा कोई भगवान है तो वह या तो शक्तिहीन या क्रूर होगा,  और उसे दोनों ही परिणाम स्वीकार नहीं करना चाहता। \n\nवक्ता का नास्तिकता होना एक आकस्मिक घटना नहीं है, बल्कि गहन अध्ययन और विचार का परिणाम है।  उन्होंने पवित्र ग्रंथों को पढ़ा, ईश्वर के अस्तित्व के पक्ष और विपक्ष में तर्कों पर विचार किया है और निष्कर्ष निकाला है कि ईश्वर में विश्वास के लिए कोई तर्कसंगत आधार नहीं है। \n\nवह मानवता में विश्वास करता है, उसके प्रगति और मुक्ति की क्षमता में।  वह मानता है कि एक बेहतर दुनिया बनाने के लिए मानवीय प्रयास, संघर्ष और बलिदान आवश्यक हैं।  उनके अनुसार 

## StuffDocumentChain Text summarization

In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("LittleListenMan.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'LittleListenMan', 'source': 'LittleListenMan.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='An  In-Depth  Look  at  Wilhelm  Reich\'s  \'Listen,  Little  Man!\'  Wilhelm  Reich\'s  "Listen,  Little  Man!"  (German:  Rede  an  den  kleinen  Mann ),  published  \nin\n \n1946,\n \nstands\n \nas\n \none\n \nof\n \nthe\n \nmost\n \nprovocative\n \nand\n \ndeeply\n \npersonal\n \nworks\n \nin\n \nthe\n \nhistory\n \nof\n \npsychology\n \nand\n \nsocial\n \ncritique.\n \nFar\n \nfrom\n \na\n \nconventional\n \nscientific\n \ntreatise,\n \nit\n \nis\n \nan\n \nimpassioned,\n \noften\n \nscathing,\n \nand\n \nultimately\n \nhopeful\n \naddress\n \nfrom\n \nthe\n \ncontroversial\n \npsychoanalyst\n \nto\n \nwhat\n \nhe\n \nterms\n \nthe\n \n"Little\n \nMan"—the\n \naverage,\n \ncommon\n \nhuman\n \nbeing.\n \nTo\n \nfully\n \ngrasp\n \nthe\n \nbook\'s\n \n

In [25]:
template=""" Write a concise and short summary of the following speech,
Speech :{text}

 """
prompt=PromptTemplate(input_variables=['text'],
                      template=template)


In [26]:
from langchain.chains.summarize import load_summarize_chain

In [27]:
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary=chain.run(docs)
output_summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and short summary of the following speech,
Speech :An  In-Depth  Look  at  Wilhelm  Reich's  'Listen,  Little  Man!'  Wilhelm  Reich's  "Listen,  Little  Man!"  (German:  Rede  an  den  kleinen  Mann ),  published  
in
 
1946,
 
stands
 
as
 
one
 
of
 
the
 
most
 
provocative
 
and
 
deeply
 
personal
 
works
 
in
 
the
 
history
 
of
 
psychology
 
and
 
social
 
critique.
 
Far
 
from
 
a
 
conventional
 
scientific
 
treatise,
 
it
 
is
 
an
 
impassioned,
 
often
 
scathing,
 
and
 
ultimately
 
hopeful
 
address
 
from
 
the
 
controversial
 
psychoanalyst
 
to
 
what
 
he
 
terms
 
the
 
"Little
 
Man"—the
 
average,
 
common
 
human
 
being.
 
To
 
fully
 
grasp
 
the
 
book's
 
intensity
 
and
 
enduring
 
relevance,
 
one
 
must
 
first
 
understand
 
the
 
tumultuous
 
life
 
and
 
radical
 
ideas
 
of
 
its
 
author.
 
1.  The  Author  and  His  Tumult

'This is an excellent summary of Wilhelm Reich\'s "Listen, Little Man!"  Here\'s a slightly more concise version, pulling out the key points:\n\n**Wilhelm Reich\'s "Listen, Little Man!" - A Summary**\n\n"Listen, Little Man!" is a powerful and controversial work by Wilhelm Reich, a psychoanalyst who broke from traditional Freudian thought. It\'s a direct address to the "average person," the "Little Man," who Reich believes is trapped in a cycle of self-oppression and societal conformity. \n\n**Key Arguments:**\n\n* **Responsibility:** Individuals are responsible for their own condition. They must critically examine their limitations and strive for self-improvement.\n* **External Authority:**  The "Little Man" seeks external authority figures instead of trusting their own judgment, leading to a cycle of disillusionment.\n* **True Freedom:** True freedom is internal, achieved through self-awareness and living authentically, not merely the absence of external constraints.\n* **The  "Little

### Map reduce to Summarize Large documents

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [29]:
docs

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'LittleListenMan', 'source': 'LittleListenMan.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='An  In-Depth  Look  at  Wilhelm  Reich\'s  \'Listen,  Little  Man!\'  Wilhelm  Reich\'s  "Listen,  Little  Man!"  (German:  Rede  an  den  kleinen  Mann ),  published  \nin\n \n1946,\n \nstands\n \nas\n \none\n \nof\n \nthe\n \nmost\n \nprovocative\n \nand\n \ndeeply\n \npersonal\n \nworks\n \nin\n \nthe\n \nhistory\n \nof\n \npsychology\n \nand\n \nsocial\n \ncritique.\n \nFar\n \nfrom\n \na\n \nconventional\n \nscientific\n \ntreatise,\n \nit\n \nis\n \nan\n \nimpassioned,\n \noften\n \nscathing,\n \nand\n \nultimately\n \nhopeful\n \naddress\n \nfrom\n \nthe\n \ncontroversial\n \npsychoanalyst\n \nto\n \nwhat\n \nhe\n \nterms\n \nthe\n \n"Little\n \nMan"—the\n \naverage,\n \ncommon\n \nhuman\n \nbeing.\n \nTo\n \nfully\n \ngrasp\n \nthe\n \nbook\'s\n \n

In [30]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'LittleListenMan', 'source': 'LittleListenMan.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='An  In-Depth  Look  at  Wilhelm  Reich\'s  \'Listen,  Little  Man!\'  Wilhelm  Reich\'s  "Listen,  Little  Man!"  (German:  Rede  an  den  kleinen  Mann ),  published  \nin\n \n1946,\n \nstands\n \nas\n \none\n \nof\n \nthe\n \nmost\n \nprovocative\n \nand\n \ndeeply\n \npersonal\n \nworks\n \nin\n \nthe\n \nhistory\n \nof\n \npsychology\n \nand\n \nsocial\n \ncritique.\n \nFar\n \nfrom\n \na\n \nconventional\n \nscientific\n \ntreatise,\n \nit\n \nis\n \nan\n \nimpassioned,\n \noften\n \nscathing,\n \nand\n \nultimately\n \nhopeful\n \naddress\n \nfrom\n \nthe\n \ncontroversial\n \npsychoanalyst\n \nto\n \nwhat\n \nhe\n \nterms\n \nthe\n \n"Little\n \nMan"—the\n \naverage,\n \ncommon\n \nhuman\n \nbeing.\n \nTo\n \nfully\n \ngrasp\n \nthe\n \nbook\'s\n \n

In [35]:
len(final_documents)


7

In [36]:
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [37]:
final_prompt='''
Provide a comprehensive final summary of Wilhelm Reich's "Listen, Little Man!" based on the provided key points.
Add a thought-provoking Title.
Start the precise summary with an introduction outlining Reich's core message to the "Little Man," and then present the key insights in numbered points.
Text:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)

In [38]:
final_prompt_template


PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide a comprehensive final summary of Wilhelm Reich\'s "Listen, Little Man!" based on the provided key points.\nAdd a thought-provoking Title.\nStart the precise summary with an introduction outlining Reich\'s core message to the "Little Man," and then present the key insights in numbered points.\nText:{text}\n\n')

In [39]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:`An  In-Depth  Look  at  Wilhelm  Reich's  'Listen,  Little  Man!'  Wilhelm  Reich's  "Listen,  Little  Man!"  (German:  Rede  an  den  kleinen  Mann ),  published  
in
 
1946,
 
stands
 
as
 
one
 
of
 
the
 
most
 
provocative
 
and
 
deeply
 
personal
 
works
 
in
 
the
 
history
 
of
 
psychology
 
and
 
social
 
critique.
 
Far
 
from
 
a
 
conventional
 
scientific
 
treatise,
 
it
 
is
 
an
 
impassioned,
 
often
 
scathing,
 
and
 
ultimately
 
hopeful
 
address
 
from
 
the
 
controversial
 
psychoanalyst
 
to
 
what
 
he
 
terms
 
the
 
"Little
 
Man"—the
 
average,
 
common
 
human
 
being.
 
To
 
fully
 
grasp
 
the
 
book's
 
intensity
 
and
 
enduring
 
relevance,
 
one
 
must
 
first
 
understand
 
the
 
tumultuous
 
life
 
and
 
radical
 
ideas
 
of
 
its
 
author.
 
1.  The  Author  and  His  Tumultuous  Context:  Wilh

Token indices sequence length is longer than the specified maximum sequence length for this model (1689 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Provide a comprehensive final summary of Wilhelm Reich's "Listen, Little Man!" based on the provided key points.
Add a thought-provoking Title.
Start the precise summary with an introduction outlining Reich's core message to the "Little Man," and then present the key insights in numbered points.
Text:This passage provides an introduction to Wilhelm Reich's book "Listen, Little Man!"  

**Here's a breakdown:**

* **The Book:**  "Listen, Little Man!" is a passionate and personal work by Reich, a controversial psychoanalyst. It's addressed to the "average person" and offers a critique of society and psychology.

* **The Author:** Wilhelm Reich was a brilliant student of Sigmund Freud but developed his own radical theories. He emphasized the importance of sexuality, orgastic potency, and the repression of natural energy in mental and societal problems.

* **Reich's Controversial Ideas:** Reich's ideas challenge

'##  The Cage of Conformity: A Look at Wilhelm Reich\'s "Listen, Little Man!"\n\nWilhelm Reich, in his fervent and personal treatise "Listen, Little Man!", directly addresses the "average person" with a scathing critique of society and its psychological underpinnings.  His core message is a call to self-awareness and responsibility, urging individuals to break free from the shackles of conformity and awaken to their true potential. \n\nHere are the key insights presented in Reich\'s work:\n\n1. **The "Emotional Plague":** Reich argues that society suffers from a pervasive neurosis characterized by repressed life energy. This emotional suppression leads to irrationality, authoritarian tendencies, and a fear of genuine feelings and freedom.\n\n2. **The "Little Man" Archetype:** Reich identifies a universal archetype, the "Little Man," who embodies the traits of the conformist, the easily manipulated individual, trapped in a cycle of suffering and subservience.  He criticizes the "Little 

### Refine Chain For Summarization

In [40]:
chain=load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)
output_summary=chain.run(final_documents)
output_summary



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"An  In-Depth  Look  at  Wilhelm  Reich's  'Listen,  Little  Man!'  Wilhelm  Reich's  "Listen,  Little  Man!"  (German:  Rede  an  den  kleinen  Mann ),  published  
in
 
1946,
 
stands
 
as
 
one
 
of
 
the
 
most
 
provocative
 
and
 
deeply
 
personal
 
works
 
in
 
the
 
history
 
of
 
psychology
 
and
 
social
 
critique.
 
Far
 
from
 
a
 
conventional
 
scientific
 
treatise,
 
it
 
is
 
an
 
impassioned,
 
often
 
scathing,
 
and
 
ultimately
 
hopeful
 
address
 
from
 
the
 
controversial
 
psychoanalyst
 
to
 
what
 
he
 
terms
 
the
 
"Little
 
Man"—the
 
average,
 
common
 
human
 
being.
 
To
 
fully
 
grasp
 
the
 
book's
 
intensity
 
and
 
enduring
 
relevance,
 
one
 
must
 
first
 
understand
 
the
 
tumultuous
 
life
 
and
 
radical
 
ideas
 
of
 
its
 
author.
 
1.  The  Author  and  His  Tumultuous  Context:  Wilhel

'Wilhelm Reich\'s "Listen, Little Man!" is a passionate and controversial work addressed to the "average" person, whom Reich refers to as "the little man."  This is not a specific individual but a universal archetype representing the masses, largely unaware of their own power and complicity in their suffering. \n\nReich, a former protégé of Sigmund Freud, diverged from orthodox psychoanalysis, focusing on character analysis and the role of sexuality and "orgone energy" in mental and societal health. He argued that the repression of natural sexual energy leads to neuroses and societal problems, and his radical ideas on sexual liberation, contraception, abortion, and "orgone energy" made him a pariah to both the psychoanalytic establishment and various political and scientific authorities. He was expelled from the International Psychoanalytic Association, fled Nazi Germany (where his books were burned), and eventually faced relentless persecution in the United States. In America, the FDA